In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()


from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
print(len(train_cands),len(dev_cands))

8272 888


In [2]:
# from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]


from snorkel.learning.utils import MentionScorer

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(len(gold_labels_dev))

Instructions for updating:
Use the retry module or similar alternatives.
296 592
888


In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [ ]:
import bz2
from six.moves.cPickle import load

with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
    ctd_unspecified, ctd_therapy, ctd_marker = load(ctd_f)

In [58]:
##### Discrete #########

def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


def LF_in_ctd_unspecified(c):
    if(cand_in_ctd_unspecified(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_therapy(c):
    if(cand_in_ctd_therapy(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_marker(c):
    if(cand_in_ctd_marker(c)==1):
        return (1,1)
    else:
        return (0,0)

import re
from snorkel.lf_helpers import (
    get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

# List to parenthetical
def ltp(x):
    return '(' + '|'.join(x) + ')'

def LF_induce(c):
    return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

causal_past = ['induced', 'caused', 'due']
def LF_d_induced_by_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
        return (1,1)
    return (0,0)

def LF_d_induced_by_c_tight(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
        return (1,1)
    return (0,0)

def LF_induce_name(c):
    return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
def LF_c_cause_d(c):
    return (1,1) if (
        re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
        and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
    ) else (0,0)

treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']
def LF_d_treat_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d(c):
    if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_treat_d(c):
    if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d_wide(c):
    if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_d(c):
    return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

def LF_c_induced_d(c):
    return (1,1) if (
        ('{{A}} {{B}}' in get_tagged_text(c)) and 
        (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
        ) else (0,0)

def LF_improve_before_disease(c):
    if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
        return (-1,1)
    return (0,0)

pat_terms = ['in a patient with ', 'in patients with']
def LF_in_patient_with(c):
    return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

uncertain = ['combin', 'possible', 'unlikely']
def LF_uncertain(c):
    if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_induced_other(c):
    if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
        return (-1,1)
    return (0,0)  

def LF_far_c_d(c):
    if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_far_d_c(c):
    if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_risk_d(c):
    if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
        return (1,1)
    return (0,0)

def LF_develop_d_following_c(c):
    return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

procedure, following = ['inject', 'administrat'], ['following']
def LF_d_following_c(c):
    return (1,1) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

def LF_measure(c):
    return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

def LF_level(c):
    return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

def LF_neg_d(c):
    return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


def LF_ctd_marker_c_d(c):
    l,s = LF_c_d(c)
    cl = cand_in_ctd_marker(c)
    return (l*cl,s*cl)

def LF_ctd_marker_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_marker(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)

def LF_ctd_therapy_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_therapy(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_unspecified(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_unspecified(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)


# def LF_ctd_marker_c_d(c):
#     return LF_c_d(c) * cand_in_ctd_marker(c)

# def LF_ctd_marker_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# def LF_ctd_therapy_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# def LF_ctd_unspecified_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# def LF_ctd_unspecified_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

def LF_closer_chem(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical closer than @dist/2 in either direction
    sent = c.get_parent()
    closest_other_chem = float('inf')
    for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    for i in range(max(0, dis_start - dist // 2), dis_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    return (0,0)

def LF_closer_dis(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical disease than @dist/8 in either direction
    sent = c.get_parent()
    for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    for i in range(max(0, chem_start - dist // 8), chem_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    return (0,0)


LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
    LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
    LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
    LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
    LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
    LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
    LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
]

LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]
print(len(LFs),len(LF_l))

In [59]:
import numpy as np

def distanceCD(c):
    dist = 0
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    return dist/5000


def distanceCD_(c,l):
    dist = []
    for w in l:
        pattern = r'({{A}})(.*)('+w+r')(.*)({{B}})'
        matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
        if(matchObj):
            match_groups = matchObj.group(2,4)
            dist.append(sum([len(mg) for mg in match_groups]))
    if(len(dist)>0):
        return min(dist)
    return 0

def distanceDC_(c,l):
    dist = []
    for w in l:
        pattern = r'({{B}})(.*)('+w+r')(.*)({{A}})'
        matchObj = re.search(pattern, get_tagged_text(c), flags=re.I)
        if(matchObj):
            match_groups = matchObj.group(2,4)
            dist.append(sum([len(mg) for mg in match_groups]))
    if(len(dist)>0):
        return min(dist)
    return 0

   

def levenshtein(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

In [ ]:
# ##### Smooth LFs #########

# def cand_in_ctd_unspecified(c):
#     return 1 if c.get_cids() in ctd_unspecified else 0

# def cand_in_ctd_therapy(c):
#     return 1 if c.get_cids() in ctd_therapy else 0

# def cand_in_ctd_marker(c):
#     return 1 if c.get_cids() in ctd_marker else 0


# def LF_in_ctd_unspecified(c):
#     if(cand_in_ctd_unspecified(c)==1):
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_in_ctd_therapy(c):
#     if(cand_in_ctd_therapy(c)==1):
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_in_ctd_marker(c):
#     if(cand_in_ctd_marker(c)==1):
#         return (1,1)
#     else:
#         return (0,0)

# import re
# from snorkel.lf_helpers import (
#     get_tagged_text,
#     rule_regex_search_tagged_text,
#     rule_regex_search_btw_AB,
#     rule_regex_search_btw_BA,
#     rule_regex_search_before_A,
#     rule_regex_search_before_B,
# )

# import re
# from snorkel.lf_helpers import (
#     get_left_tokens, get_right_tokens, get_between_tokens,
#     get_text_between, get_tagged_text,
# )

# # List to parenthetical
# def ltp(x):
#     return '(' + '|'.join(x) + ')'

# # def LF_induce(c):
# #     return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# def LF_induce(c):
#     return (1,distanceCD_(c,['induc'])) if re.search(r'{{A}}.*induc.*{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# causal_past = ['induced', 'caused', 'due']
# # def LF_d_induced_by_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
# #         return (1,1)
# #     return (0,0)

# def LF_d_induced_by_c(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_between_tokens(c))
#     for w in causal_past:
#         sc=max(sc,get_similarity(word_vectors,w))
#     return (1,sc)

# # def LF_d_induced_by_c_tight(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
# #         return (1,1)
# #     return (0,0)

# def LF_d_induced_by_c_tight(c):
#     if (rule_regex_search_btw_BA(c, '.*' + ltp(causal_past) + ' (by|to) ', 1)==1):
#         return (1,(1-distanceDC_(c,causal_past)))
#     return (0,0)

# def LF_induce_name(c):
#     return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

# causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
# # def LF_c_cause_d(c):
# #     return (1,1) if (
# #         re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
# #         and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
# #     ) else (0,0)


# def LF_c_cause_d(c):
#     return (1,(1-distanceCD_(c,causal))) if (
#         re.search(r'{{A}}.* ' + ltp(causal) + '.*{{B}}', get_tagged_text(c), re.I)
#         and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
#     ) else (0,0)


# treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']
# # def LF_d_treat_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_d_treat_c(c):
#     if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceDC_(c,treat))
#     return (0,0)


# # def LF_c_treat_d(c):
# #     if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_c_treat_d(c):
#     if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

# # def LF_treat_d(c):
# #     if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_treat_d(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     for w in treat:
#         sc=max(sc,get_similarity(word_vectors,w))
#     if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
#         return (0,0)
#     else:
#         return (-1,sc)

# # def LF_c_treat_d_wide(c):
# #     if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_c_treat_d_wide(c):
#     if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
#         return (-1,1-distanceCD_(c,treat))
#     return (0,0)

# def LF_c_d(c):
#     return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

# def LF_c_induced_d(c):
#     return (1,1) if (
#         ('{{A}} {{B}}' in get_tagged_text(c)) and 
#         (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
#         ) else (0,0)

# # def LF_improve_before_disease(c):
# #     if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
# #         return (-1,1)
# #     return (0,0)


# def distanceImproveBeforeDisease(c):
#     m=re.search(r'(improv)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return len(m.group(2))/5000
#     return 0


# def LF_improve_before_disease(c):
#     if(rule_regex_search_before_B(c, 'improv.*', -1) == -1):
#         return (-1,1-distanceImproveBeforeDisease(c))
#     else:
#         return (0,0)


# pat_terms = ['in a patient with ', 'in patients with']
# def LF_in_patient_with(c):
#     return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

# uncertain = ['combin', 'possible', 'unlikely']
# # def LF_uncertain(c):
# #     if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_uncertain(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     for w in uncertain:
#         sc=max(sc,get_similarity(word_vectors,w))
#     if(re.search('(not|no|none) .* {{B}}', get_tagged_text(c), re.I)):
#         return (0,0)
#     else:
#         return (-1,sc)

# # def LF_induced_other(c):
# #     if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)  

# def LF_induced_other(c):
#     if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)  

# # def LF_far_c_d(c):
# #     if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_far_c_d(c):
#     if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)

# # def LF_far_d_c(c):
# #     if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
# #         return (-1,1)
# #     return (0,0)

# def LF_far_d_c(c):
#     if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
#         return (-1,distanceCD(c))
#     return (0,0)

# #without deps
# gen_model.weights.lf_accuracy
# # def LF_risk_d(c):
# #     if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
# #         return (1,1)
# #     return (0,0)


# def LF_risk_d(c):
#     sc = 0
#     word_vectors = get_word_vectors(get_left_tokens(c[1],7))
#     sc=max(sc,get_similarity(word_vectors,'risk'))
#     return (1,sc)


# # def LF_develop_d_following_c(c):
# #     return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceDevFol(c):
#     dist = 0
#     matchObj = re.search(r'(develop)(.*)({{B}})(.*)(following)(.*)({{A}})', get_tagged_text(c), flags=re.I)
#     if(matchObj):
#         match_groups = matchObj.group(2,4,6)
#         dist = sum([len(mg) for mg in match_groups])
#     return dist/5000

# def LF_develop_d_following_c(c):
#     return (1,1-distanceDevFol(c)) if re.search(r'develop.*{{B}}.*following.*{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# procedure, following = ['inject', 'administrat'], ['following']
# # def LF_d_following_c(c):
# #     return (1,distanceDFollC(c)) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)


# def LF_d_following_c(c):
#     return (1,1-distanceDC_(c,following)) if re.search('{{B}}.*' + ltp(following) + '.*{{A}}.*' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

# # def LF_measure(c):
# #     return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceMeasureA(c):
#     m = re.search('(measur)(.*)({{A}})', get_tagged_text(c), flags=re.I) 
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_measure(c):
#     return (-1,distanceMeasureA(c)) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)


# # def LF_level(c):
# #     return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceLevel(c):
#     m = re.search('({{A}})(.*)(level)', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0

# def LF_level(c):
#     return (-1,distanceLevel(c)) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

# # def LF_neg_d(c):
# #     return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# def distanceNeg(c):
#     m = re.search('(none|not|no)(.*)({{B}})', get_tagged_text(c), flags=re.I)
#     if(m):
#         return (5000-len(m.group(2)))/5000
#     return 0


# def LF_neg_d(c):
#     return (-1,distanceNeg(c)) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)


# WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
#                 'seems', 'suggests', 'risk', 'implicated',
#                'the aim', 'to (investigate|assess|study)']

# WEAK_RGX = r'|'.join(WEAK_PHRASES)

# def LF_weak_assertions(c):
#     return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


# def LF_ctd_marker_c_d(c):
#     l,s = LF_c_d(c)
#     cl = cand_in_ctd_marker(c)
#     return (l*cl,s*cl)

# def LF_ctd_marker_induce(c):
#     l1,s1 = LF_c_induced_d(c)
#     l2,s2 = LF_d_induced_by_c_tight(c)
#     cl = cand_in_ctd_marker(c)
#     return ((l1 or l2)*cl,max(s1,s2)*cl)

# def LF_ctd_therapy_treat(c):
#     l,s = LF_c_treat_d_wide(c)
#     cl = cand_in_ctd_therapy(c)
#     return (l*cl,s*cl)

# def LF_ctd_unspecified_treat(c):
#     l,s = LF_c_treat_d_wide(c)
#     cl = cand_in_ctd_unspecified(c)
#     return (l*cl,s*cl)

# def LF_ctd_unspecified_induce(c):
#     l1,s1 = LF_c_induced_d(c)
#     l2,s2 = LF_d_induced_by_c_tight(c)
#     cl = cand_in_ctd_unspecified(c)
#     return ((l1 or l2)*cl,max(s1,s2)*cl)


# # def LF_ctd_marker_c_d(c):
# #     return LF_c_d(c) * cand_in_ctd_marker(c)

# # def LF_ctd_marker_induce(c):
# #     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# # def LF_ctd_therapy_treat(c):
# #     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# # def LF_ctd_unspecified_treat(c):
# #     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# # def LF_ctd_unspecified_induce(c):
# #     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

# def LF_closer_chem(c):
#     # Get distance between chemical and disease
#     chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
#     dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
#     if dis_start < chem_start:
#         dist = chem_start - dis_end
#     else:
#         dist = dis_start - chem_end
#     # Try to find chemical closer than @dist/2 in either direction
#     sent = c.get_parent()
#     closest_other_chem = float('inf')
#     for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
#             return (-1,1)
#     for i in range(max(0, dis_start - dist // 2), dis_start):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
#             return (-1,1)
#     return (0,0)

# def LF_closer_dis(c):
#     # Get distance between chemical and disease
#     chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
#     dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
#     if dis_start < chem_start:
#         dist = chem_start - dis_end
#     else:
#         dist = dis_start - chem_end
#     # Try to find chemical disease than @dist/8 in either direction
#     sent = c.get_parent()
#     for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Disease' and cid != sent.entity_cids[dis_start]:
#             return (-1,1)
#     for i in range(max(0, chem_start - dist // 8), chem_start):
#         et, cid = sent.entity_types[i], sent.entity_cids[i]
#         if et == 'Disease' and cid != sent.entity_cids[dis_start]:
#             return (-1,1)
#     return (0,0)


# LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
#     LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
#     LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
#     LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
#     LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
#     LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
#     LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
# ]

# LF_l = [
#     1,1,1,-1,-1,-1,
#     -1,1,1,-1,
#     -1,1,1,
#     1,1,-1,1,
#     -1,-1,-1,-1,
#     1,-1,1,1,-1,
#     -1,-1,-1,1,-1,-1,-1
# ]
# print(len(LFs),len(LF_l))

In [49]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [ ]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)

np.save("dev_L_S_smooth",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_smooth",np.array(train_L_S))

print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [4]:
LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]

import numpy as np
dev_L_S = np.load("dev_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")

test_L_S = dev_L_S
true_labels = gold_labels_dev
print(dev_L_S.shape,train_L_S.shape)

(888, 2, 33) (8272, 2, 33)


In [5]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [6]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

33


In [11]:
## normalized model with smooth LFs

def train_nl_s(lr,ep,th,af):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
#         print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),\
                           -tf.ones_like(v))
#             print("out",out)
            return out

        ## smooth pout
        pout = tf.map_fn(lambda c: iskequalsy(l,c)*s_ ,np.array([-1,1],dtype=np.float64),name="pout")
       
        ## discrete pout
#         pout = tf.map_fn(lambda c: iskequalsy(l,c) ,np.array([-1,1],dtype=np.float64),name="pout")

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,name="t_pout")
#         print("pout",pout)

#         print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
#         print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
#             print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
#             print("intsy",out1)
            return out1
        
        ## discrete normalizer
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),np.arange(NoOfClasses,dtype=np.float64))


        ## smooth normalizer
        zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                   np.array([-1,1],dtype=np.float64),name="zy")
    

#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
#         print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
#         print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
#         print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))


        normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
        
   
        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
#         print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)

                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [12]:
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    predicted_labels=train_nl_s(0.01/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,seed),\
                           tf.truncated_normal_initializer(i,0.001,seed))

alpha-mean 0.0
0 loss 276454.1542314165
[ 0.01055597  0.00460971  0.00741015  0.00990061  0.01086975  0.00967792
  0.01159471  0.00715229  0.00835285  0.01053219  0.00969638  0.00806058
  0.00982088 -0.0061938   0.01019854  0.0103459   0.00846706  0.00885507
  0.01025639  0.00833013  0.00925665 -0.00385336  0.00993405  0.00818247
  0.00472511  0.01175482  0.01061927  0.01065399  0.01052143 -0.00712766
  0.00622718  0.00639786  0.00923408]
[[1.10806905 0.80955813 1.00379715 0.99072586 1.09655594 1.03263284
  1.15937217 1.02605476 1.14074346 1.05483374 0.97370305 1.06042058
  0.97239845 1.08446464 1.03100092 1.03231293 0.84348392 0.91025144
  1.04993045 0.8264619  1.02720751 0.9456143  0.98625403 1.1650302
  0.86956646 1.16750624 1.05818307 1.05855864 1.04755632 0.99645537
  0.81445804 0.83079723 0.95282736]]
{0: 263, 1: 625}
acc 0.5146396396396397
(0.392, 0.8277027027027027, 0.5320304017372421, None)

1 loss 272332.1897618076
[ 0.01996494  0.01118729  0.01482879  0.01977512  0.02066303 

In [10]:
predicted_labels=train_nl_s(0.1/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,seed),\
                           tf.truncated_normal_initializer(0.7,0.001,seed))

0 loss 151207.9510220813
[0.80989634 0.67315788 0.68017891 0.78008226 0.78229886 0.7209384
 0.73905133 0.6777094  0.68313512 0.78145473 0.77970013 0.68498014
 0.81359542 0.6657093  0.8115127  0.78098404 0.74263401 0.77820079
 0.78134898 0.76310889 0.71415314 0.64373272 0.76641697 0.67630578
 0.67339027 0.78366627 0.75220351 0.75261417 0.73701073 0.67243256
 0.72924456 0.73752678 0.71826069]
[[1.019613   0.84043425 1.03121076 0.91986869 1.02490664 1.02395617
  1.14220693 1.05554699 1.16671749 0.98351553 0.90294608 1.08432899
  0.88295845 1.11028577 0.94208126 0.96118012 0.81351358 0.83984537
  0.97845085 0.77795277 1.02472561 0.99854039 0.93630415 1.18525401
  0.90046597 1.09560349 1.0256237  1.02858404 1.03157643 0.99170345
  0.80039274 0.79720836 0.94816149]]
acc 0.5168918918918919
(0.4008941877794337, 0.9087837837837838, 0.5563598759048605, None)

1 loss 149935.3607799438
[0.90312327 0.65495563 0.66771003 0.75200622 0.77560989 0.74365941
 0.75377123 0.66211804 0.67236441 0.76624852 0

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [83]:
# for i in np.linspace(0,1,11):
#     print("alpha-mean",i)
#     predicted_labels=train_nl_s(0.001/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12),\
#                            tf.truncated_normal_initializer(i,0.1,12))
    
predicted_labels=train_nl_s(0.001/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12),\
                           tf.truncated_normal_initializer(0.3,0.1,12))

0 loss 177593.6089042592
[0.41854278 0.11520687 0.31047011 0.30153524 0.40736582 0.34136772
 0.47013164 0.33035577 0.44548349 0.36387167]
[[1.11655652 0.81322315 1.00867167 0.99972798 1.10542697 1.04178828
  1.16826229 1.03197667 1.14706397 1.06550782]]
acc 0.9068941009239516
(0.26143790849673204, 0.21164021164021163, 0.23391812865497075, None)

1 loss 177396.56970193452
[0.41954094 0.11619926 0.31138325 0.30244885 0.40835387 0.3413951
 0.47105875 0.32949968 0.44462813 0.36301599]
[[1.11556877 0.8122317  1.00778122 0.99882951 1.10447602 1.04224007
  1.16731925 1.0327495  1.14779659 1.06629769]]
acc 0.9086709310589908
(0.2702702702702703, 0.21164021164021163, 0.2373887240356083, None)

2 loss 177199.12666951108
[0.42053909 0.11719167 0.3122979  0.30336394 0.409342   0.34142099
 0.47198704 0.32864074 0.44376979 0.36215742]
[[1.11458114 0.81124022 1.00688941 0.99792959 1.10352504 1.04269297
  1.16637726 1.03352561 1.14853279 1.06709081]]
acc 0.9136460554371002
(0.29850746268656714, 0.2116

In [82]:
predicted_labels=train_nl_s(0.1/len(train_L_S),5,\
                            tf.truncated_normal_initializer(1,0.1,seed),\
                            tf.truncated_normal_initializer(0.9,0.001,seed))

0 loss 155264.53954891075
[0.92690352 0.9971321  0.89117171 0.89079329 0.90543897 0.98363892
 0.9542917  0.89181359 0.8937952  0.91162108]
[[1.10009277 0.72704278 1.03085428 1.02047423 1.10633782 0.96621163
  1.12864769 1.05197377 1.16535794 1.05758622]]
acc 0.8941009239516702
(0.33126934984520123, 0.5661375661375662, 0.41796875, None)

1 loss 154352.82584004718
[0.94538075 1.00003227 0.88239876 0.88170429 0.90954794 0.99722517
 0.98051568 0.88340816 0.88622291 0.92065953]
[[1.0884174  0.72649249 1.05094155 1.03924606 1.10629312 0.95459477
  1.11214221 1.07165552 1.18340155 1.05207841]]
acc 0.8948116560056859
(0.3333333333333333, 0.5661375661375662, 0.41960784313725485, None)

2 loss 154345.657013675
[0.95711568 1.00017134 0.87422725 0.87406136 0.91291265 0.99776011
 0.99031907 0.87564378 0.87869541 0.92818664]
[[1.08146765 0.72632277 1.06995853 1.0570634  1.10656614 0.94945574
  1.10669314 1.09036948 1.20057795 1.04758399]]
acc 0.8941009239516702
(0.3333333333333333, 0.576719576719576

In [81]:
predicted_labels=train_nl_s(0.1/len(train_L_S),5,\
                            tf.truncated_normal_initializer(1,0.1,seed),\
                            tf.truncated_normal_initializer(0.9,0.01,seed))

0 loss 155548.10499263214
[0.93326346 0.98037827 0.88953763 0.88821575 0.91311962 0.98554735
 0.96406621 0.89759127 0.90964637 0.91662847]
[[1.1032539  0.72197787 1.03206361 1.02162306 1.10767904 0.96823482
  1.13417441 1.05126276 1.16661986 1.05815618]]
acc 0.8855721393034826
(0.3147632311977716, 0.5978835978835979, 0.41240875912408764, None)

1 loss 154383.652287932
[0.94953699 1.00003838 0.8767362  0.87537624 0.91583377 0.99209049
 0.98585481 0.89384701 0.90618009 0.92479421]
[[1.09328744 0.7103856  1.05400886 1.04201962 1.10870954 0.92510664
  1.12145815 1.06944685 1.18536492 1.05320904]]
acc 0.8855721393034826
(0.3147632311977716, 0.5978835978835979, 0.41240875912408764, None)

2 loss 154346.07266892138
[0.96029182 1.00012677 0.86560251 0.86458486 0.91794999 0.98825132
 0.99431731 0.89279564 0.90488922 0.93167133]
[[1.08722855 0.71028976 1.07575335 1.06207778 1.11012153 0.84748815
  1.11739958 1.08540677 1.20247734 1.04934008]]
acc 0.8781094527363185
(0.3121951219512195, 0.6772486

In [14]:
## normalized model with smooth LFs + penalties

def train(lr,ep,th,af,plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0,p3k=3):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## plv : all possible label values = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
#         print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
#             print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: iskequalsy(l,c)*s_ ,plv,name="pout")
        else:
            pout = tf.map_fn(lambda c: iskequalsy(l,c) ,plv,name="pout")

#         print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
#         print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
#         print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
#             print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
#             print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           plv,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           plv,name="zy")

    
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
#         print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
#         print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
#         print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))

        if(penalty == 1):
            normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))
        elif(penalty == 2):
            normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)
        elif(penalty == 3):
             normloss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk)))
        else:
             normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
   
        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [15]:
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=1)


alpha-mean 0.0
0 loss 258761.97377532846
[ 0.09760699  0.06906155  0.07856466  0.09640611  0.09653547  0.09181254
  0.09855085  0.07455734  0.07604416  0.09698834  0.09583047  0.07974186
  0.10121241 -0.07568579  0.10015734  0.09719328  0.09931358  0.09386556
  0.09543832  0.09541713  0.08916486 -0.03168285  0.09725393  0.07409003
  0.06457212  0.09928821  0.09774973  0.09802521  0.09776105 -0.07706665
  0.08102752  0.08125664  0.09399042]
[[1.02183361 0.76346911 0.94839377 0.90496805 1.01156448 0.95517189
  1.07388363 0.97576553 1.08656111 0.9691725  0.8882573  1.00127791
  0.88250813 1.14733461 0.9423475  0.94631146 0.7545906  0.82570798
  0.96529566 0.74009843 0.95249652 0.98549148 0.89985832 1.18311283
  0.82864581 1.08103604 0.97220814 0.97224309 0.96137654 1.06089836
  0.74204982 0.75805573 0.87081834]]
{0: 167, 1: 721}
acc 0.48761261261261263
(0.3897364771151179, 0.9493243243243243, 0.5526057030481809, None)

1 loss 223371.00639377302
[ 0.19647516  0.1519857   0.16648469  0.190

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.3333333333333333, 1.0, 0.5, None)

alpha-mean 0.1
0 loss 234151.0249429247
[0.19806045 0.17301418 0.18142486 0.19625912 0.19624729 0.19126569
 0.19848645 0.17789195 0.17930346 0.19683213 0.19564832 0.18255269
 0.20088443 0.0270292  0.20009142 0.19709826 0.19905759 0.19369319
 0.19528246 0.1954141  0.18820886 0.0759723  0.1972427  0.17741569
 0.16916418 0.19927373 0.19771028 0.19800886 0.19771434 0.02561138
 0.17918176 0.17941287 0.1936894 ]
[[1.02197974 0.76605671 0.95114066 0.90567352 1.01230799 0.95745409
  1.07473267 0.97857589 1.08884084 0.96988075 0.88897396 1.00345416
  0.88307601 1.14544416 0.94277432 0.94700273 0.75525368 0.82633383
  0.96588505 0.74078055 0.9551535  0.98543526 0.9005408  1.18350274
  0.83097049 1.08170613 0.97296341 0.9729511  0.96213915 1.05915984
  0.74376368 0.7596745  0.87238416]]
{0: 164, 1: 724}
acc 0.48873873873873874
(0.39088397790055246, 0.956081081081081, 0.5549019607843136, None)

1 loss 202293.79076913805
[ 0.29633312  0.25646121  0.26945875  0.

In [16]:
for i in np.linspace(0,1,11):
    print()
    print("alpha-mean",i)
    train(0.1/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(i,0.001,seed),
                          plv=np.array([-1,1],dtype=np.float64),smooth=True,penalty=2)


alpha-mean 0.0
0 loss 258761.97377532846
[ 0.09760699  0.06906155  0.07856466  0.09640611  0.09653547  0.09181254
  0.09855085  0.07455734  0.07604416  0.09698834  0.09583047  0.07974186
  0.10121241 -0.07568579  0.10015734  0.09719328  0.09931358  0.09386556
  0.09543832  0.09541713  0.08916486 -0.03168285  0.09725393  0.07409003
  0.06457212  0.09928821  0.09774973  0.09802521  0.09776105 -0.07706665
  0.08102752  0.08125664  0.09399042]
[[1.02183361 0.76346911 0.94839377 0.90496805 1.01156448 0.95517189
  1.07388363 0.97576553 1.08656111 0.9691725  0.8882573  1.00127791
  0.88250813 1.14733461 0.9423475  0.94631146 0.7545906  0.82570798
  0.96529566 0.74009843 0.95249652 0.98549148 0.89985832 1.18311283
  0.82864581 1.08103604 0.97220814 0.97224309 0.96137654 1.06089836
  0.74204982 0.75805573 0.87081834]]
{0: 167, 1: 721}
acc 0.48761261261261263
(0.3897364771151179, 0.9493243243243243, 0.5526057030481809, None)

1 loss 223371.00639377302
[ 0.19647516  0.1519857   0.16648469  0.190

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0 loss 234151.0249429247
[0.19806045 0.17301418 0.18142486 0.19625912 0.19624729 0.19126569
 0.19848645 0.17789195 0.17930346 0.19683213 0.19564832 0.18255269
 0.20088443 0.0270292  0.20009142 0.19709826 0.19905759 0.19369319
 0.19528246 0.1954141  0.18820886 0.0759723  0.1972427  0.17741569
 0.16916418 0.19927373 0.19771028 0.19800886 0.19771434 0.02561138
 0.17918176 0.17941287 0.1936894 ]
[[1.02197974 0.76605671 0.95114066 0.90567352 1.01230799 0.95745409
  1.07473267 0.97857589 1.08884084 0.96988075 0.88897396 1.00345416
  0.88307601 1.14544416 0.94277432 0.94700273 0.75525368 0.82633383
  0.96588505 0.74078055 0.9551535  0.98543526 0.9005408  1.18350274
  0.83097049 1.08170613 0.97296341 0.9729511  0.96213915 1.05915984
  0.74376368 0.7596745  0.87238416]]
{0: 164, 1: 724}
acc 0.48873873873873874
(0.39088397790055246, 0.956081081081081, 0.5549019607843136, None)

1 loss 202293.79076913805
[ 0.29633312  0.25646121  0.26945875  0.29061716  0.28913602  0.28385634
  0.29442709  0.2632

In [54]:
## normalized model with discrete LFs

def train_nl(lr,ep,th):
    
    BATCH_SIZE = 1
    tf.reset_default_graph()


    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
        print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.3,0.1,seed),\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

        print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
        print(alphas)
        print(s)
        print("l",l)
        print(s.graph)

        s_ = tf.maximum(tf.subtract(s,alphas), 0)
        print("s_",s_)

       
    
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),\
                           -tf.ones_like(v))
            print("out",out)
            return out

        ### discrete pout 
        pout = tf.map_fn(lambda c: iskequalsy(l,c) ,np.array([-1,1],dtype=np.float64),name="pout")
       
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,name="t_pout")
        print("pout",pout)

        print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            print("intsy",out1)
            return out1
        
        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),np.arange(NoOfClasses,dtype=np.float64))

        
        print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
        print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))

        
        normloss = tf.negative(tf.reduce_sum(lsp  - logz  ))


        tf.summary.scalar('un-normloss', normloss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
        print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
        print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
                tl = 0
                try:
                    it = 0
                    while True:
                        sm,_,ls,t = sess.run([summary_merged,train_step,normloss,thetas])
#                         print(t)
#                         print(tl)
                        train_writer.add_summary(sm, it)
#                         if(ls<1e-5):
#                             break
                        tl = tl + ls
                        it = it + 1
                        
                except tf.errors.OutOfRangeError:
                    pass
                print(en,"loss",tl)

                sess.run(dev_init_op)
                sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                test_writer.add_summary(sm, en)
#                 print(a)
#                 print(t)

#                 unique, counts = np.unique(pl, return_counts=True)
#                 print(dict(zip(unique, counts)))
                print("acc",accuracy_score(true_labels,pl))
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(true_labels,pl))

            predictAndPrint(pl)
            print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

#             cf = confusion_matrix(true_labels,pl)
#             print(cf)
    return pl

In [55]:
predicted_labels=train_nl(0.1/len(train_L_S),5,tf.truncated_normal_initializer(1,0.1,12))

next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
k Tensor("Const:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
out Tensor("pout/while/Select:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("map/while/Select:0", shape=(10,), dtype=float64)
zy Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=floa